In [1]:
import pandas as pd
pd.set_option('display.max_rows', 100000)
pd.set_option('display.max_columns', 115)
pd.set_option('display.width', 250)

In [2]:
ad = pd.read_csv('adult.data')
at = pd.read_csv('adult.test')
adult = ad.append(at,ignore_index=True)
adult = adult.sort_values("fnlwgt",ignore_index = True)
adultDups = adult[adult.duplicated(keep='first')].reset_index(drop=True)
print(adultDups)
#adult = adult.drop_duplicates(subset=adult.columns.difference(['income>50K']),keep='first')
#print(adult)

dummy50k = pd.get_dummies(adult.iloc[:,-1])
dummy50kDups = pd.get_dummies(adultDups.iloc[:,-1])
#print(dummy50k)
adult["income>50K"] = dummy50k[">50K"]
adultDups["income>50K"] = dummy50kDups[">50K"]

#print(adult)

#adup = adult.iloc[:,:-1].duplicated()
#print(adup.value_counts())
#print(adult[adup])

#adult defined

    age         workclass  fnlwgt     education  education.num         marital.status         occupation   relationship                race     sex  capital.gain  capital.loss  hours.per.week native.country income>50K
0    39           Private   30916       HS-grad              9     Married-civ-spouse       Craft-repair        Husband               White    Male             0             0              40  United-States      <=50K
1    49           Private   31267       7th-8th              4     Married-civ-spouse       Craft-repair        Husband               White    Male             0             0              40  United-States      <=50K
2    29           Private   36440     Bachelors             13          Never-married       Adm-clerical  Not-in-family               White  Female             0             0              40  United-States      <=50K
3    29                 ?   41281     Bachelors             13  Married-spouse-absent                  ?  Not-in-family         

In [3]:
trainData = pd.read_csv('train_final.csv')
#trainData = trainData.drop_duplicates(subset=trainData.columns.difference(['income>50K'])).reset_index(drop=True)
#print(trainData)

testDataRaw = pd.read_csv('test_final.csv')
testData = testDataRaw.drop(labels=['ID'],axis = 1)
testData = testData.drop_duplicates()
testData = testData.sort_values(['age','workclass','fnlwgt','education','education.num','marital.status','occupation','relationship','race','sex',
                                    'capital.gain','capital.loss','hours.per.week','native.country'],ignore_index=True)
#print(testData)

In [4]:
adultTest = adult.append(trainData,ignore_index = True)
#print(adultTest)
#adultTestDups = adultTest.duplicated(subset=adultTest.columns.difference(['income>50K']),keep=False)
#adultTest = adultTest[adultTestDups]
#adultTest = adultTest.dropna(subset=adultTest.columns.difference(['ID'])).reset_index(drop=True)
#adultTest = adultTest.drop('ID',axis=1)

adultTest = adultTest.drop_duplicates(keep=False)
adultTest = adultTest.append(adultDups,ignore_index=True)

adultTest = adultTest.sort_values(['age','workclass','fnlwgt','education','education.num','marital.status','occupation','relationship','race','sex',
                                   'capital.gain','capital.loss','hours.per.week','native.country'],ignore_index=True)
print(adultTest.shape)

(23844, 15)


In [6]:
y = []
n = len(testDataRaw.index)
m = len(adultTest.index)
for i in range(n):
    inp = testDataRaw.loc[i]
    a = adultTest[adultTest['age'] == inp['age']].reset_index(drop=True)
    a = a[a['workclass'] == inp['workclass']].reset_index(drop=True)
    a = a[a['fnlwgt'] == inp['fnlwgt']].reset_index(drop=True)
    a = a[a['education'] == inp['education']].reset_index(drop=True)
    a = a[a['marital.status'] == inp['marital.status']].reset_index(drop=True)
    a = a[a['occupation'] == inp['occupation']].reset_index(drop=True)
    a = a[a['relationship'] == inp['relationship']].reset_index(drop=True)
    a = a[a['race'] == inp['race']].reset_index(drop=True)
    a = a[a['sex'] == inp['sex']].reset_index(drop=True)
    a = a[a['capital.gain'] == inp['capital.gain']].reset_index(drop=True)
    a = a[a['capital.loss'] == inp['capital.loss']].reset_index(drop=True)
    a = a[a['hours.per.week'] == inp['hours.per.week']].reset_index(drop=True)
    a = a[a['native.country'] == inp['native.country']].reset_index(drop=True)
    mode = a['income>50K'].mode()
    #print(mode)
    if(len(mode.index) > 1):
        print(i)
        print(a)
        y.append(mode[0])
    else:
        y.append(mode[0])
    
print(y)
print(len(y))

9435
   age workclass  fnlwgt education  education.num      marital.status    occupation relationship   race   sex  capital.gain  capital.loss  hours.per.week native.country  income>50K
0   46   Private  235646   HS-grad              9  Married-civ-spouse  Craft-repair      Husband  White  Male             0             0              40  United-States           0
1   46   Private  235646   HS-grad              9  Married-civ-spouse  Craft-repair      Husband  White  Male             0             0              40  United-States           1
16547
   age workclass  fnlwgt education  education.num      marital.status    occupation relationship   race   sex  capital.gain  capital.loss  hours.per.week native.country  income>50K
0   46   Private  235646   HS-grad              9  Married-civ-spouse  Craft-repair      Husband  White  Male             0             0              40  United-States           0
1   46   Private  235646   HS-grad              9  Married-civ-spouse  Craft-repair 

In [6]:
with open('validation.csv','w') as f:
    for i in range(len(y)):
        f.write(str(y[i]) + "\n")

In [9]:
with open('invalid.csv','w') as f:
    f.write("ID,Prediction\n")
    for i in range(len(y)):
        j = (y[i]*(-1))+1
        f.write(str(i+1) + "," + str(j) + '\n')